In [29]:
%reload_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 100

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, offline
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

import requests

from os import listdir
import unicodedata

from paths import *
from scripts.manipulation import remove_acentos
from scripts.manipulation import normalize_cols
from scripts import scrap_data
from scripts import manipulation
from scripts import io
from scripts import vis_graphs

from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')

import yaml

import geopandas as gpd

# Taubate

- Nao temos acesso aos dados por municipio dos seguintes indicadores: 
 - Ocupação leitos UTI COVID
 - Leitos COVID/100 mil hab
 - Internações

In [30]:
sp_full = pd.read_csv('https://raw.githubusercontent.com/seade-R/dados-covid-sp/master/data/dados_covid_sp.csv', sep=';')
sp_full['nome_drs'] = np.where(sp_full['nome_munic']=='São Paulo','Município de São Paulo',sp_full['nome_drs'])




In [31]:
cols = ['datahora','nome_munic','codigo_ibge','casos','casos_novos','obitos','obitos_novos','nome_drs']

mask = sp_full['nome_drs'].isin(['Taubaté'])
vale = sp_full[mask][cols]


rename_cols = {
    "casos":"casos_tbt",
    "casos_novos":"casos_novos_tbt",
    "obitos":"obitos_tbt",
    "obitos_novos":"obitos_novos_tbt",
}
vale_ = vale.groupby(by=['nome_drs','datahora'], as_index=False).sum()

vale_['nome_munic'] = 'Vale'
vale_['codigo_ibge'] = 0


vale = pd.concat([vale,vale_], axis=0)

vale_ = vale_.rename(columns = rename_cols).drop(columns=['nome_munic','codigo_ibge'])
vale = vale.merge(vale_, on=['nome_drs','datahora'])

In [32]:
cols = ['casos','obitos','casos_tbt','obitos_tbt']


dd_final_all = pd.DataFrame()

for municipio in vale['nome_munic'].unique():
    tbt = vale[vale['nome_munic']==municipio]

    for col in cols:
        

        tbt[f'{col}_shift_7'] = tbt[f'{col}'].shift(7)
        tbt[f'{col}_shift_14'] = tbt[f'{col}'].shift(14)


        tbt[f'{col}_7d'] = tbt[f'{col}'] - tbt[f'{col}_shift_7']
        tbt[f'{col}_14d'] = tbt[f'{col}'] - tbt[f'{col}_shift_14']


        tbt[f'{col}_shift_14'] = tbt[f'{col}_7d'].shift(7)

        tbt[f'{col}_var'] = tbt[f'{col}_7d']/tbt[f'{col}_shift_14']
        
        tbt = tbt.drop(columns = [f'{col}_shift_14',f'{col}_shift_7',])



#     dd = tbt[['datahora','nome_munic',f'{col}',f'{col}_7',f'{col}_14d',f'{col}_var']]

    dd_final_all = pd.concat([dd_final_all,tbt], axis=0)


    


vale_final = dd_final_all.copy()

In [33]:
vale_final['casos_tbt'] = 100 * vale_final['casos'] / vale_final['casos_tbt'] 
vale_final['casos_novos_tbt'] = 100 * vale_final['casos_novos'] / vale_final['casos_novos_tbt'] 
vale_final['obitos_tbt'] = 100 * vale_final['obitos'] / vale_final['obitos_tbt'] 
vale_final['obitos_novos_tbt'] = 100 * vale_final['obitos_novos'] / vale_final['obitos_novos_tbt'] 
vale_final = vale_final.fillna(0)

In [34]:
vale_final = vale_final.sort_values(by=['datahora','nome_munic'])
vale_final['codigo_ibge'] = vale_final['codigo_ibge'].astype(int)
vale_final['datahora'] = pd.to_datetime(vale_final['datahora'])

In [35]:
vale_final.tail()

,datahora,nome_munic,codigo_ibge,casos,casos_novos,obitos,obitos_novos,nome_drs,casos_tbt,casos_novos_tbt,obitos_tbt,obitos_novos_tbt,casos_7d,casos_14d,casos_var,obitos_7d,obitos_14d,obitos_var,casos_tbt_7d,casos_tbt_14d,casos_tbt_var,obitos_tbt_7d,obitos_tbt_14d,obitos_tbt_var
6514,2020-08-05,São Sebastião,3550704,969,26,15,1,Taubaté,5.098122,5.949657,2.626970,6.25,146.0,332.0,0.784946,1.0,8.0,0.142857,3114.0,6960.0,0.809672,63.0,157.0,0.670213
6516,2020-08-05,Taubaté,3554102,1664,24,55,0,Taubaté,8.754669,5.491991,9.632224,0.00,234.0,731.0,0.470825,1.0,8.0,0.142857,3114.0,6960.0,0.809672,63.0,157.0,0.670213
6517,2020-08-05,Tremembé,3554805,106,2,0,0,Taubaté,0.557689,0.457666,0.000000,0.00,9.0,31.0,0.409091,0.0,0.0,0.000000,3114.0,6960.0,0.809672,63.0,157.0,0.670213
6518,2020-08-05,Ubatuba,3555406,570,8,20,0,Taubaté,2.998895,1.830664,3.502627,0.00,99.0,223.0,0.798387,0.0,4.0,0.000000,3114.0,6960.0,0.809672,63.0,157.0,0.670213
6519,2020-08-05,Vale,0,19007,437,571,16,Taubaté,100.000000,100.000000,100.000000,100.00,3114.0,6960.0,0.809672,63.0,157.0,0.670213,3114.0,6960.0,0.809672,63.0,157.0,0.670213


### RT

In [36]:
#load br cities
geo_sp = gpd.read_file('../../brasil_geodata/maps/sp_municipios.json')
# cols = ['geocodigo','nome_mesorregiao','geometry']
cols = ['geocodigo','nome_mesorregiao','nome_municipio']
geo_sp = geo_sp[cols]

In [43]:
ds['state_num_id'].unique()

array([11, 12, 13, 14, 15, 16, 17, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31,
       32, 33, 35, 41, 42, 43, 50, 51, 52, 53])

In [48]:
#load state data
url  = 'http://datasource.coronacidades.org/'
data = 'br/states/rt'

ds = pd.read_csv(f'{url}{data}')

ds['last_updated'] = pd.to_datetime(ds['last_updated'])

sp = ds[ds['state_num_id']==35]
sp = sp.rename(columns={'state_num_id':'city_id'})
sp['city_id'] = 'SP'
sp['nome_mesorregiao'] = 'SP'
sp['nome_municipio']   = 'Estado de SP'

# rj = ds[ds['state']=='RJ']
# rj = rj.rename(columns={'state':'city_id'})
# rj['nome_mesorregiao'] = 'RJ'
# rj['nome_municipio']   = 'Estado do RJ'

# mg = ds[ds['state']=='MG']
# mg = mg.rename(columns={'state':'city_id'})
# mg['nome_mesorregiao'] = 'MG'
# mg['nome_municipio']   = 'Estado de MG'

# am = ds[ds['state']=='AM']
# am = am.rename(columns={'state':'city_id'})
# am['nome_mesorregiao'] = 'AM'
# am['nome_municipio']   = 'Estado de AM'

In [49]:
#load cities data
data = 'br/cities/rt'

df = pd.read_csv(f'{url}{data}')

df['last_updated'] = pd.to_datetime(df['last_updated'])


In [50]:
dd = df.merge(geo_sp, how='right', left_on = 'city_id', right_on = 'geocodigo')
dd.head()


regiao = 'Vale do Paraíba Paulista'
mask = (dd['nome_mesorregiao']==regiao)
vale = dd[mask]
vale = vale[vale['city_id'].notnull()]

mask = (dd['nome_municipio']=='São Paulo')
sp_city = dd[mask]


vale = pd.concat([vale,sp_city], 0)
vale = pd.concat([vale,sp], 0)

vale['geocodigo'] = vale['geocodigo'].fillna(111).astype(int)

In [51]:
vale = vale.rename(columns={'geocodigo':'codigo_ibge','last_updated':'datahora'})
vale['datahora'] = pd.to_datetime(vale['datahora'])

In [52]:
mask = vale['datahora']== max(vale['datahora'])
vale_rt_last = vale[mask]
vale_rt_last = vale_rt_last.drop(columns=['datahora'])
vale_rt_last = vale_rt_last.rename(columns={'nome_municipio':'municipio'})

In [53]:
mask = vale_final['nome_munic']=='Vale'
just_vale = vale_final[mask]
just_vale = just_vale.drop(columns=['nome_munic'])
just_vale['last_update'] = max(just_vale['datahora'])

In [54]:
#upload to drive
from scripts import io
io.to_gbq(vale_final,'vale_covid','covid','gabinete-sv', if_exists='replace')
io.to_gbq(just_vale,'just_vale_covid','covid','gabinete-sv', if_exists='replace')
io.to_gbq(vale,'rt_vale_covid','covid','gabinete-sv', if_exists='replace')
io.to_gbq(vale_rt_last,'vale_rt_last','covid','gabinete-sv', if_exists='replace')
io.to_gbq(just_vale,'just_vale_covid','covid','gabinete-sv', if_exists='replace')

1it [00:05,  5.06s/it]
1it [00:04,  4.13s/it]
1it [00:05,  5.16s/it]
1it [00:04,  4.89s/it]
1it [00:03,  3.31s/it]


## Plano SP

In [ ]:
sp_full = pd.read_csv('https://raw.githubusercontent.com/seade-R/dados-covid-sp/master/data/dados_covid_sp.csv', sep=';')
sp_full['nome_drs'] = np.where(sp_full['nome_munic']=='São Paulo','Município de São Paulo',sp_full['nome_drs'])

cols = ['datahora','nome_munic','casos','obitos','nome_drs']
sp = sp_full[cols][sp_full['nome_drs'].notnull()]

sp = sp.groupby(by=['nome_drs','datahora'], as_index=False).sum().sort_values(by=['nome_drs','datahora'])

estado = sp.groupby(by=['datahora'], as_index=False).sum().sort_values(by=['datahora'])
estado['nome_drs'] = 'Estado de São Paulo'
estado = estado[sp.columns]

sp = pd.concat([sp,estado],axis=0)

In [ ]:
sp['casos'] = sp['casos'].astype(int)
sp['obitos'] = sp['obitos'].astype(int)

In [ ]:
sp['nome_drs'].unique()

In [ ]:
dd_final_all = pd.DataFrame()

for drs in sp['nome_drs'].unique():

    tbt = sp[sp['nome_drs']==drs]

    tbt['casos_shift'] = tbt['casos'].shift(7)
    tbt['obitos_shift'] = tbt['obitos'].shift(7)
    tbt['obitos_shift_14'] = tbt['obitos'].shift(14)


    tbt['casos_7'] = tbt['casos'] - tbt['casos_shift']
    tbt['obitos_7'] = tbt['obitos'] - tbt['obitos_shift']
    tbt['obitos_14d'] = tbt['obitos'] - tbt['obitos_shift_14']


    tbt['casos_7_shift'] = tbt['casos_7'].shift(7)
    tbt['obitos_7_shift'] = tbt['obitos_7'].shift(7)

    tbt['casos_var'] = tbt['casos_7']/tbt['casos_7_shift']
    tbt['obitos_var'] = tbt['obitos_7']/tbt['obitos_7_shift']


        
        
    dd = tbt[['datahora','nome_drs','casos_var','obitos_var','obitos_14d']]

    dd_final_all = pd.concat([dd_final_all,dd], axis=0)


    


dd_final = dd_final_all.copy()

## Internacoes

In [ ]:
sp_cv = pd.read_csv('https://github.com/seade-R/dados-covid-sp/raw/master/data/plano_sp_leitos_internacoes.csv', sep=';')
sp_cv['internacoes_var'] = sp_cv['internacoes_7d']/sp_cv['internacoes_7d_l']
sp_cv['internacoes_14d'] = sp_cv['internacoes_7d'].astype(int) + sp_cv['internacoes_7d_l'].astype(int)
cols = ['datahora','nome_drs','leitos_pc','internacoes_var','internacoes_14d','pop']

uti_all = sp_cv[cols]

uti_all['nome_drs'] = uti_all['nome_drs'].apply(lambda x: x[7:] if x[:3]=='DRS' else x).str.strip()



df_final = uti_all[cols].copy()

In [ ]:
sp_cv.head()

In [ ]:
df_final.tail()

In [ ]:
dd_final.tail()

## Ocupacao UTI

In [ ]:
uti = pd.read_csv('../data/sp_gov/uti_ocupacao.csv', sep=';')



uti_final = uti.copy()




# mask =  uti_final['datahora'] == max(uti_final['datahora'])

# uti_final = uti_final[mask]

# uti_final['datahora'] = max(df_final['datahora'])

In [ ]:

df = pd.merge(dd_final,df_final,on=['datahora','nome_drs'],how='outer').merge(uti_final,on=['datahora','nome_drs'],how='inner')
mask = df['nome_drs'].str.startswith('Grande')
df = df[np.logical_not(mask)]
df['leitos_pc'] = df['leitos_pc'].str.replace(',','.')
df['pop'] = df['pop'].astype(int)

df['internacoes_14d_pc'] = (df['internacoes_14d'] * 10**5 )/df['pop']
df['obitos_14d_pc'] = (df['obitos_14d']  * 10**5 )/df['pop']

## Add Fases

In [ ]:

df = pd.merge(dd_final,df_final,on=['datahora','nome_drs'],how='outer').merge(uti_final,on=['datahora','nome_drs'],how='inner')
mask = df['nome_drs'].str.startswith('Grande')
df = df[np.logical_not(mask)]
df['leitos_pc'] = df['leitos_pc'].str.replace(',','.')

df['pop'] = df['pop'].astype(int)

df['internacoes_14d_pc'] = (df['internacoes_14d'] * 10**5 )/df['pop']
df['obitos_14d_pc'] = (df['obitos_14d']  * 10**5 )/df['pop']

df = df.round(2)

def uti_fase(x):
    if x>=80:
        fase=1
    elif (x <80) & (x>=75):
        fase=2
    else:
        fase = 4
    return fase

def leitos_fase(x):
    if x<=3:
        fase=1
    elif (x >3) & (x<=5):
        fase=2
    else:
        fase = 4
    return fase


O = df['uti_var'].apply(lambda x: uti_fase(x))

L = df['leitos_pc'].str.replace(',','.').astype(float).apply(lambda x: leitos_fase(x))


c1 = (O*4 + L)/5

df['Capacidade Hospitalar'] = c1
# df['Capacidade Hospitalar'] = c1.astype(int)



def casos_fase(x):
    if x>=2:
        fase=1
    elif (x <2) & (x>=1):
        fase=3
    else:
        fase = 4
    return fase

def internacoes_fase(x,y):
    if x>=1.5:
        fase=1
    elif (x <1.5) & (x>=1):
        fase=2
    elif (x <1) & (y>=40):
        fase=3
    elif (x <1) & (y<40):
        fase=4
    return fase

def obitos_fase(x,y):
    if x>=2:
        fase=1
    elif (x <2) & (x>=1):
        fase=2
    elif (x <1) & (y>=5):
        fase=3
    elif (x <1) & (y<5):
        fase=4
    return fase

NC = df['casos_var'].apply(lambda x: casos_fase(x))

NI = df.apply(lambda x: internacoes_fase(x['internacoes_var'],x['internacoes_14d_pc']), axis=1)

NO = df.apply(lambda x: obitos_fase(x['obitos_var'],x['obitos_14d_pc']), axis=1)

c2 = (NC + 3*NI + NO)/5

df['Evolução da Pandemia'] = c2

df['casos_var'] = df['casos_var'].round(2).astype(str)
df['obitos_var'] = df['obitos_var'].round(2).astype(str)
df['internacoes_var'] = df['internacoes_var'].round(2).astype(str)
df['uti_var'] = df['uti_var'].round(2).astype(str)

df['Classif. Final'] = df[['Capacidade Hospitalar','Evolução da Pandemia']].min(axis=1)

# df['Classif. Final'] = df['Classif. Final'].astype(int).astype(str)
# df['Evolução da Pandemia'] = df['Evolução da Pandemia'].astype(int).astype(str)
# df['Capacidade Hospitalar'] = df['Capacidade Hospitalar'].astype(int).astype(str)

df['Classif. Final'] = df['Classif. Final'].round(1).astype(str)
df['Evolução da Pandemia'] = df['Evolução da Pandemia'].round(1).astype(str)
df['Capacidade Hospitalar'] = df['Capacidade Hospitalar'].round(1).astype(str)

df['internacoes_14d_pc'] = df['internacoes_14d_pc'].round(1).astype(str)
df['obitos_14d_pc'] = df['obitos_14d_pc'].round(1).astype(str)


## Padronize 

In [27]:

cols = ['datahora','nome_drs','Capacidade Hospitalar','uti_var','leitos_pc','Evolução da Pandemia','casos_var','internacoes_var','internacoes_14d_pc','obitos_var','obitos_14d_pc','Classif. Final']

cols = ['datahora','nome_drs','Capacidade Hospitalar','uti_var','leitos_pc','Evolução da Pandemia','casos_var','internacoes_var','obitos_var','Classif. Final']



rename_drs = {
    'Araraquara':'DRS 03 - Araraquara',
    'Araçatuba':'DRS 02 - Araçatuba',
    'Baixada Santista':'DRS 04 - Baixada Santista',
    'Barretos':'DRS 05 - Barretos',
    'Bauru':'DRS 06 - Bauru',
    'Campinas':'DRS 07 - Campinas',
    'Estado de São Paulo':'Estado de São Paulo',
    'Franca':'DRS 08 - Franca',
    'Marília':'DRS 09 - Marília',
    'Município de São Paulo':'DRS 01 - Município de São Paulo',
    'Piracicaba':'DRS 10 - Piracicaba',
    'Presidente Prudente':'DRS 11 - Presidente Prudente',
    'Registro':'DRS 12 - Registro',
    'Ribeirão Preto':'DRS 13 - Ribeirão Preto',
    'Sorocaba':'DRS 16 - Sorocaba',
    'São José do Rio Preto':'DRS 15 - São José do Rio Preto',
    'São João da Boa Vista':'DRS 14 - São João da Boa Vista',
    'Taubaté':'DRS 17 - Taubaté',
    'Estado de São Paulo':'00 - Estado de São Paulo'
}

df['nome_drs'] = df['nome_drs'].map(rename_drs)

ddf = df.copy()


In [28]:
cols_rename = {
    "Capacidade Hospitalar": "cap_hosp",
    "Evolução da Pandemia": "evolucao_pand",
    "Classif. Final": "classif_final",
    "nome_drs": "drs_name",
}


ddf = ddf.rename(columns=cols_rename)
mask = ddf['datahora']==max(ddf['datahora'])
ddf['last_update'] = max(ddf['datahora'])
ddf_last = ddf[mask].rename(columns={"drs_name":"DRS"})

#upload to drive
from scripts import io
io.to_gbq(ddf,'plano_sp','covid','gabinete-sv', if_exists='replace')
io.to_gbq(ddf_last,'plano_sp_last','covid','gabinete-sv', if_exists='replace')

1it [00:05,  5.26s/it]
1it [00:06,  6.97s/it]
